# Experimental Data Analysis

## Run init

We start by importing the modules for this initing and loading the csv artifact from the previous component into a pandas dataframe:

In [1]:
import wandb
import pandas as pd

In [2]:
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)

wandb: Currently logged in as: cadmus (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

## Profiling

Using pandas profiling, we profile the data to check for correctness and for any issues:

In [4]:
import pandas_profiling

In [5]:
profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Below we fix the outlier problem with the `price` feature, and convert the `last_review` column to `datetime` type:

In [6]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

Below we can confirm that `last_review` is now indeed `datetime`.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  nu

Below we can also confirm that the `price` feature is bound between 10 and 350.

In [8]:
df.price.describe()

count    19001.000000
mean       122.340456
std         71.530346
min         10.000000
25%         66.000000
50%        100.000000
75%        160.000000
max        350.000000
Name: price, dtype: float64

In [9]:
run.finish()